In [ ]:
#importando bibliotecas essenciais
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import warnings
import itertools
warnings.filterwarnings("ignore") # specify to ignore warning messages

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


> Importando os dados


### Norte

In [ ]:
df_norte = pd.read_csv('https://raw.githubusercontent.com/luk3mn/TCC-predicao-de-series-temporais/master/dataset/pld_norte.csv')
# Removendo todos as virgulas do dataset
df_norte.replace(',','.', regex=True, inplace=True)

# Convertendo a coluna "Data" em datetime
df_norte['inicio_semana'] = pd.to_datetime(df_norte['inicio_semana'])

# Tranformando a coluna "Data" em index
df_norte.set_index('inicio_semana', inplace=True)

# Convertendo todos os objetos para valores numericos
c = df_norte.select_dtypes(object).columns
df_norte[c] = df_norte[c].apply(pd.to_numeric,errors='coerce')
df_norte.head()

,preco_pesado,preco_medio,preco_leve,geracao_hidraulica,geracao_termica,carga_energia,reservatorio_EARM,reservatorio_ENA
inicio_semana,,,,,,,,
2001-08-04,684.0,684.0,684.0,2952.29,0.0,2204.00,62.42,89.57
2001-08-11,684.0,684.0,684.0,2956.57,0.0,2141.57,59.75,88.57
2001-08-18,684.0,684.0,684.0,2909.71,0.0,2110.43,56.75,83.86
2001-08-25,684.0,684.0,684.0,2631.00,0.0,2095.43,53.63,79.29
2001-09-01,684.0,684.0,684.0,2558.86,0.0,2069.14,50.64,76.00


### Nordeste

In [ ]:
df_nordeste = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Previsões de séries temporais/teste_com_o_tcc/dataset/pld_nordeste.xlsx')
df_nordeste['inicio_semana'] = pd.to_datetime(df_nordeste['inicio_semana'], dayfirst=True)
df_nordeste.sort_values('inicio_semana', inplace=True)
df_nordeste.set_index('inicio_semana', inplace=True)
df_nordeste.head()

,preco_pesado,preco_medio,preco_leve,geracao_hidraulica,geracao_termica,carga_energia,reservatorio_EARM,reservatorio_ENA
inicio_semana,,,,,,,,
2001-08-04,684.0,684.0,684.0,29242.57,3921.71,33446.00,20.09,60.14
2001-08-11,684.0,684.0,684.0,28742.71,4210.86,33554.00,19.22,59.86
2001-08-18,684.0,684.0,684.0,28801.29,4251.00,33838.14,23.78,62.71
2001-08-25,684.0,684.0,684.0,28746.00,4390.43,34118.86,17.29,58.43
2001-09-01,684.0,684.0,684.0,28657.14,4191.86,33868.00,16.26,59.14


### Sul

In [ ]:
df_sul = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Previsões de séries temporais/teste_com_o_tcc/dataset/pld_sul.xlsx')
df_sul['inicio_semana'] = pd.to_datetime(df_sul['inicio_semana'], dayfirst=True)
df_sul.sort_values('inicio_semana', inplace=True)
df_sul.set_index('inicio_semana', inplace=True)
df_sul.head()

,preco_pesado,preco_medio,preco_leve,geracao_hidraulica,geracao_termica,carga_energia,reservatorio_EARM,reservatorio_ENA
inicio_semana,,,,,,,,
2001-08-04,684.0,684.0,684.0,29242.57,3921.71,33446.00,20.09,60.14
2001-08-11,684.0,684.0,684.0,28742.71,4210.86,33554.00,19.22,59.86
2001-08-18,684.0,684.0,684.0,28801.29,4251.00,33838.14,23.78,62.71
2001-08-25,684.0,684.0,684.0,28746.00,4390.43,34118.86,17.29,58.43
2001-09-01,684.0,684.0,684.0,28657.14,4191.86,33868.00,16.26,59.14


### Sudeste

In [ ]:
df_sudeste = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Previsões de séries temporais/teste_com_o_tcc/dataset/pld_sudeste.xlsx')
df_sudeste['inicio_semana'] = pd.to_datetime(df_sudeste['inicio_semana'], dayfirst=True)
df_sudeste.sort_values('inicio_semana', inplace=True)
df_sudeste.set_index('inicio_semana', inplace=True)
df_sudeste.head()

,preco_pesado,preco_medio,preco_leve,geracao_hidraulica,geracao_termica,carga_energia,reservatorio_EARM,reservatorio_ENA
inicio_semana,,,,,,,,
2001-08-04,684.0,684.0,684.0,29242.57,3921.71,33446.00,20.09,60.14
2001-08-11,684.0,684.0,684.0,28742.71,4210.86,33554.00,19.22,59.86
2001-08-18,684.0,684.0,684.0,28801.29,4251.00,33838.14,23.78,62.71
2001-08-25,684.0,684.0,684.0,28746.00,4390.43,34118.86,17.29,58.43
2001-09-01,684.0,684.0,684.0,28657.14,4191.86,33868.00,16.26,59.14


## Encontrando os parâmetros para a base de dados do PLD

In [ ]:
def find_params(data, p_min, p_max, q_min, q_max, d_max):
    # Define the p, d and q parameters to take any value between 0 and 2
    p = range(p_min, (p_max+1))
    d = range(0, (d_max+1))
    q = range(q_min, (q_max+1))

    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p, d, q))

    aic=[]
    for param in pdq:
        try:
            mod = ARIMA(data, order=param)
            results = mod.fit(disp=0)
            aic.append(results.aic)
            # pega o melhor modelo
            if (np.min(aic) == results.aic):
                min_aic=results.aic
                best_model=param
            print('ARIMA{} - AIC:{}'.format(param, results.aic))
        except:
            continue
    print("\n========================================")
    print("             MELHOR MODELO               ")
    print("----------------------------------------")
    print('ARIMA{} - AIC:{}'.format(best_model, min_aic))
    print("========================================")

## Encontrando os parâmetros no dataset PLD Norte

In [ ]:
find_params(df_norte['preco_medio'],0,9,0,9,3)

ARIMA(0, 0, 0) - AIC:5469.449825236253
ARIMA(0, 0, 1) - AIC:5005.486868110417
ARIMA(0, 1, 0) - AIC:4366.051478335235
ARIMA(0, 1, 1) - AIC:4358.976659621956
ARIMA(0, 1, 2) - AIC:4359.194093223194
ARIMA(0, 1, 3) - AIC:4359.795591114114
ARIMA(0, 1, 4) - AIC:4356.248627070969
ARIMA(0, 1, 5) - AIC:4354.456026148959
ARIMA(0, 1, 6) - AIC:4355.318611412953
ARIMA(0, 1, 7) - AIC:4350.767383390851
ARIMA(0, 1, 8) - AIC:4336.0345342392875
ARIMA(0, 1, 9) - AIC:4335.98940598577
ARIMA(0, 2, 0) - AIC:4600.365564023802
ARIMA(0, 2, 1) - AIC:4363.3352028493
ARIMA(0, 2, 2) - AIC:4356.449390192005
ARIMA(1, 0, 0) - AIC:4380.359052095902
ARIMA(1, 0, 1) - AIC:4370.99999093259
ARIMA(1, 0, 2) - AIC:4372.39174452298
ARIMA(1, 0, 3) - AIC:4374.027712792201
ARIMA(1, 0, 4) - AIC:4371.869887289171
ARIMA(1, 0, 5) - AIC:4370.623379927911
ARIMA(1, 0, 6) - AIC:4371.207351749637
ARIMA(1, 0, 7) - AIC:4366.280360504887
ARIMA(1, 0, 8) - AIC:4351.954598910074
ARIMA(1, 0, 9) - AIC:4351.871510842799
ARIMA(1, 1, 0) - AIC:4360.285